In [ ]:
from gGA.data import block_to_feature
from ase.io import read
from gGA.data import AtomicData
from gGA.utils.make_kpoints import kmesh_sampling
import torch
from gGA.nn.ghostG import GhostGutzwiller

torch.set_default_dtype(torch.float64)
U = 1.5
J = 0. # 0.25 * U
Up = U - 2*J
Jp = J

gga = GhostGutzwiller(
    atomic_number=torch.tensor([6]),
    nocc={"C":[2]},
    basis={"C":[2]},
    idx_intorb={"C":[0]},
    naux=3,
    intparams={"C":[{"U":U,"Up":Up,"J":J, "Jp":Jp}]},
    nspin=1, 
    device=torch.device("cpu"),
    kBT=0.002,
)

block = {
    "0_0_0_0_0": 0.2*torch.diag(torch.tensor([1.,-1.])),
    "0_0_0_0_1": -torch.eye(2)*(1/6),
    "0_0_0_1_0": -torch.eye(2)*(1/6),
    "0_0_1_0_0": -torch.eye(2)*(1/6)
}


# hop = torch.diag(torch.tensor([0.8,1.2,-1.2,-0.8]))
# block = {
#     "0_0_0_0_0": 0.2*torch.diag(torch.tensor([0.8,1.2,-1.2,-0.8])),
#     "0_0_0_0_1": -hop*(1/6),
#     "0_0_0_1_0": -hop*(1/6),
#     "0_0_1_0_0": -hop*(1/6)
# }


atomicdata = AtomicData.from_ase(
    read("/nessa/users/zhanghao/dev/Hubbard/gGA/test/C_cube.vasp"),
    r_max=3.1
    )

atomicdata["kpoint"] = torch.tensor(kmesh_sampling([10,10,10], True)).to(torch.get_default_dtype())
block_to_feature(atomicdata, gga.kinetic.idp_phy, block)
atomicdata = AtomicData.to_AtomicDataDict(atomicdata)

import matplotlib.pyplot as plt

errs = []
for i in range(100):
    # print(gga.gGAtomic.LAM_C["C"][1].abs().max())
    # err, RDM, RDM_fit, LAMS = gga.update(atomicdata)
    err, RDM = gga.update(atomicdata)
    print("\t ## Err of current loop {:d}: {:.4f}, lam_emb: {:.4f}, N_emb: {:.2f}".format(i+1, err, gga.gGAtomic.LAM_C["C"][0].abs().max().item(), RDM["C"][0].trace()))# , RDM_fit["C"][0].trace()))
    # print(torch.linalg.eigvalsh(RDM["C"][0]))
    errs.append(err)

    # print(gga.gGAtomic.LAM_C)
    # print(RDM["C"][0])
    # print(err)
    # print(torch.linalg.eigvalsh(RDM["C"][0]), RDM["C"][0].trace())

plt.matshow(RDM["C"][0].detach(), cmap="bwr", vmax=1.0, vmin=-1.0)
plt.show()
    # DM_kin:  tensor([0.1600, 0.1600, 0.8400, 0.8400])

Find E_fermi: 0.0003
DM_kin:  tensor([1.6946e-04, 1.6946e-04, 5.9777e-03, 5.9777e-03, 5.0202e-01, 5.0202e-01,
        5.0618e-01, 5.0618e-01, 9.8903e-01, 9.8903e-01, 9.9663e-01, 9.9663e-01])
DM_emb:  tensor([3.7746e-06, 3.7746e-06, 1.1719e-03, 1.1719e-03, 3.2639e-02, 3.2639e-02,
        3.0075e-01, 3.0075e-01, 9.9789e-01, 9.9789e-01, 9.9996e-01, 9.9996e-01])
	 ## Err of current loop 1: 0.7874, lam_emb: 0.6598, N_emb: 4.66
Find E_fermi: 0.3898
DM_kin:  tensor([5.7161e-04, 5.7161e-04, 7.7881e-03, 7.7881e-03, 2.2410e-01, 2.2410e-01,
        7.6780e-01, 7.6780e-01, 9.9980e-01, 9.9980e-01, 9.9994e-01, 9.9994e-01])
DM_emb:  tensor([2.0618e-04, 2.0618e-04, 3.2960e-03, 3.2960e-03, 6.9579e-02, 6.9579e-02,
        4.0041e-01, 4.0041e-01, 9.9762e-01, 9.9762e-01, 9.9948e-01, 9.9948e-01])
	 ## Err of current loop 2: 0.1548, lam_emb: 1.0357, N_emb: 4.94
Find E_fermi: 0.6502
DM_kin:  tensor([6.8826e-04, 6.8826e-04, 1.0442e-02, 1.0442e-02, 1.7039e-01, 1.7039e-01,
        8.1950e-01, 8.1950e-01, 9.9934

KeyboardInterrupt: 

In [2]:
torch.linalg.eigvalsh(gga.gGAtomic.LAM["C"][0])

_LinAlgError: linalg.eigh: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated eigenvalues (error code: 10).